<a href="https://colab.research.google.com/github/Gjeffroy/LLM_finetuning/blob/main/Preparing_training_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Basics

In [ ]:
! pip install datasets
! pip install pandas
! pip install openai
! pip install mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.6/254.6 kB 6.7 MB/s eta 0:00:00


In [ ]:
# Replace with your actual API key
from google.colab import userdata

import os
from openai import OpenAI

client = OpenAI(
    # This is the default and can be omitted
    api_key=userdata.get('openai_API_key'),
)


In [ ]:
from datasets import load_dataset

ds = load_dataset("louisbrulenaudet/code-urbanisme")

In [ ]:
ds['train']

Dataset({
    features: ['ref', 'title_main', 'texte', 'dateDebut', 'dateFin', 'num', 'id', 'cid', 'type', 'etat', 'nota', 'version_article', 'ordre', 'conditionDiffere', 'infosComplementaires', 'surtitre', 'nature', 'texteHtml', 'dateFinExtension', 'versionPrecedente', 'refInjection', 'idTexte', 'idTechInjection', 'origine', 'dateDebutExtension', 'idEliAlias', 'cidTexte', 'sectionParentId', 'multipleVersions', 'comporteLiensSP', 'sectionParentTitre', 'infosRestructurationBranche', 'idEli', 'sectionParentCid', 'numeroBo', 'infosRestructurationBrancheHtml', 'historique', 'infosComplementairesHtml', 'renvoi', 'fullSectionsTitre', 'notaHtml', 'inap'],
    num_rows: 2417
})

In [ ]:
import pandas as pd

df = pd.DataFrame(ds['train'])

In [ ]:
template = """Cite moi l'article {article}"""
df['article_id'] = df['ref'].str.split(',').str[-1].str.replace('art.', '')
df['instruction'] = df['article_id'].map(lambda x: template.format(article=x))
df[['article_id', 'instruction', 'texte']]

,article_id,instruction,texte
0,L101-1,Cite moi l'article L101-1,Le territoire français est le patrimoine commu...
1,L101-2,Cite moi l'article L101-2,Dans le respect des objectifs du développement...
2,L101-2-1,Cite moi l'article L101-2-1,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Cite moi l'article L101-3,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Cite moi l'article L102-1,L'autorité administrative compétente de l'Etat...
...,...,...,...
2412,A614-2,Cite moi l'article A614-2,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Cite moi l'article A614-3,Les dépenses correspondants à la rémunération ...
2414,A614-4,Cite moi l'article A614-4,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Cite moi l'article A431-10 Annexe,Attestation du contrôleur technique établissan...


In [ ]:
def generate_response(prompt):
  chat_completion = client.chat.completions.create(
      messages=[
          {
              "role": "user",
              "content": prompt,
          }
      ],
      model="gpt-3.5-turbo",
  )
  return chat_completion.choices[0].message.content

#### Questions orientées (spécifique à l'article)

In [ ]:
prompt = """
Imagine une question dont la reponse se trouve dans le TEXTE suivant

TEXTE: \\n {}
"""

In [ ]:
test = prompt.format(df['texte'][0])
test

"\nImagine une question dont la reponse se trouve dans le TEXTE suivant\n\nTEXTE: \\n Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.\n"

In [ ]:
# test on the first 5 item of df
for i in range(5):
  print('TEXTE: ' + df['texte'][i])
  print(generate_response(prompt.format(df['texte'][i])))

TEXTE: Le territoire français est le patrimoine commun de la nation. Les collectivités publiques en sont les gestionnaires et les garantes dans le cadre de leurs compétences. En vue de la réalisation des objectifs définis à l'article L. 101-2 , elles harmonisent leurs prévisions et leurs décisions d'utilisation de l'espace dans le respect réciproque de leur autonomie.
Quels sont les organismes responsables de la gestion du territoire français ?
TEXTE: Dans le respect des objectifs du développement durable, l'action des collectivités publiques en matière d'urbanisme vise à atteindre les objectifs suivants : 1° L'équilibre entre : a) Les populations résidant dans les zones urbaines et rurales ; b) Le renouvellement urbain, le développement urbain et rural maîtrisé, la restructuration des espaces urbanisés, la revitalisation des centres urbains et ruraux, la lutte contre l'étalement urbain ; c) Une utilisation économe des espaces naturels, la préservation des espaces affectés aux activité

In [ ]:
# df['question_orientée'] = df['texte'].map(lambda x: generate_response(prompt.format(x)))

In [ ]:
df[['article_id', 'question_orientée', 'texte']]

,article_id,question_orientée,texte
0,L101-1,Quels acteurs sont désignés comme gestionnaire...,Le territoire français est le patrimoine commu...
1,L101-2,Quels sont les objectifs du développement dura...,Dans le respect des objectifs du développement...
2,L101-2-1,Quels sont les critères à prendre en compte po...,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Quels sont les territoires exclus de la réglem...,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Quelles sont les deux conditions requises pour...,L'autorité administrative compétente de l'Etat...
...,...,...,...
2412,A614-2,Quel est le montant de la rémunération au titr...,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Quels sont les crédits sur lesquels sont imput...,Les dépenses correspondants à la rémunération ...
2414,A614-4,Quelles sont les autorités compétentes pour mo...,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Qu'est-ce que le contrôleur technique atteste ...,Attestation du contrôleur technique établissan...


In [ ]:
# save df
df.to_csv('df_questions_orientees.csv', index=False)

### Réponses orientés

#### OpenAI

In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('openai_API_key')

In [ ]:
import pandas as pd
# load df
df = pd.read_csv('df_questions_orientees.csv')

In [ ]:
prompt = """
Repond à la QUESTION en utilisant le TEXTE suivant. La réponse doit être factuelle et uniquement basée sur le TEXTE.
Si la reponse ne se trouve pas dans le TEXTE, renvoi une string vide.

QUESTION:\\n {} \\n
TEXTE: \\n {}

Renvoi juste la reponse sans la question ni le texte.
"""

In [ ]:
from openai import AsyncOpenAI
import openai
import asyncio
from tqdm.notebook import tqdm

client_async = AsyncOpenAI()
client = openai.OpenAI()

# Limit to 500 requests per minute
RATE_LIMIT = 500
TIME_FRAME = 120  # seconds

def generate_response(prompt):
  messages = [{"role": "user", "content": prompt}]
  chat_completion = client.chat.completions.create(
      messages=messages,
      model="gpt-3.5-turbo",
  )
  return chat_completion.choices[0].message.content

In [ ]:
df[['article_id', 'question_orientée', 'reponse_orientée', 'texte']]

,article_id,question_orientée,reponse_orientée,texte
0,L101-1,Quels acteurs sont désignés comme gestionnaire...,Deux mois,Le territoire français est le patrimoine commu...
1,L101-2,Quels sont les objectifs du développement dura...,Trois mois,Dans le respect des objectifs du développement...
2,L101-2-1,Quels sont les critères à prendre en compte po...,un bordereau de dépôt,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Quels sont les territoires exclus de la réglem...,13407,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Quelles sont les deux conditions requises pour...,trois mois.,L'autorité administrative compétente de l'Etat...
...,...,...,...,...
2412,A614-2,Quel est le montant de la rémunération au titr...,,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Quels sont les crédits sur lesquels sont imput...,,Les dépenses correspondants à la rémunération ...
2414,A614-4,Quelles sont les autorités compétentes pour mo...,,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Qu'est-ce que le contrôleur technique atteste ...,,Attestation du contrôleur technique établissan...


In [ ]:
# count empty string
df[['article_id', 'question_orientée', 'reponse_orientée', 'texte']].apply(lambda x: x.str.len()).apply(lambda x: x == 0).sum()

,0
article_id,0
question_orientée,0
reponse_orientée,2305
texte,0


#### MistralAI

In [ ]:
import pandas as pd
# load df
df = pd.read_csv('df_reponse_orientees_mistral.csv')

In [ ]:
df.head(-5)

,ref,title_main,texte,dateDebut,dateFin,num,id,cid,type,etat,...,historique,infosComplementairesHtml,renvoi,fullSectionsTitre,notaHtml,inap,article_id,instruction,question_orientée,reponse_ministral_3b
0,"Code de l'urbanisme, art. L101-1",Code de l'urbanisme,Le territoire français est le patrimoine commu...,1451606400000,32472144000000,L101-1,LEGIARTI000031210068,LEGIARTI000031210068,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie législative &gt; Livre Ier : Réglementa...,NaN,NaN,L101-1,Cite moi l'article L101-1,Quels acteurs sont désignés comme gestionnaire...,Les collectivités publiques sont les gestionna...
1,"Code de l'urbanisme, art. L101-2",Code de l'urbanisme,Dans le respect des objectifs du développement...,1703030400000,32472144000000,L101-2,LEGIARTI000048599920,LEGIARTI000031210070,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie législative &gt; Livre Ier : Réglementa...,NaN,NaN,L101-2,Cite moi l'article L101-2,Quels sont les objectifs du développement dura...,Les objectifs du développement durable visés p...
2,"Code de l'urbanisme, art. L101-2-1",Code de l'urbanisme,L'atteinte des objectifs mentionnés au 6° bis ...,1629849600000,32472144000000,L101-2-1,LEGIARTI000043967077,LEGIARTI000043967077,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie législative &gt; Livre Ier : Réglementa...,NaN,NaN,L101-2-1,Cite moi l'article L101-2-1,Quels sont les critères à prendre en compte po...,Les critères à prendre en compte pour atteindr...
3,"Code de l'urbanisme, art. L101-3",Code de l'urbanisme,La réglementation de l'urbanisme régit l'utili...,1451606400000,32472144000000,L101-3,LEGIARTI000031210072,LEGIARTI000031210072,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie législative &gt; Livre Ier : Réglementa...,NaN,NaN,L101-3,Cite moi l'article L101-3,Quels sont les territoires exclus de la réglem...,Les territoires exclus de la réglementation de...
4,"Code de l'urbanisme, art. L102-1",Code de l'urbanisme,L'autorité administrative compétente de l'Etat...,1451606400000,32472144000000,L102-1,LEGIARTI000031210078,LEGIARTI000031210078,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie législative &gt; Livre Ier : Réglementa...,NaN,NaN,L102-1,Cite moi l'article L102-1,Quelles sont les deux conditions requises pour...,Le projet doit être destiné à l'une des activi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2407,"Code de l'urbanisme, art. A520-8",Code de l'urbanisme,Le défaut d'opposition par le redevable au tit...,222393600000,32472144000000,A520-8,LEGIARTI000006814211,LEGIARTI000006814211,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie réglementaire - Arrêtés &gt; Livre V :...,NaN,NaN,A520-8,Cite moi l'article A520-8,Quelles conséquences a le défaut d'opposition ...,NaN
2408,"Code de l'urbanisme, art. A520-9",Code de l'urbanisme,Le ministre chargé de l'urbanisme ou son délég...,222393600000,32472144000000,A520-9,LEGIARTI000006814212,LEGIARTI000006814212,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie réglementaire - Arrêtés &gt; Livre V :...,NaN,NaN,A520-9,Cite moi l'article A520-9,A qui doit-on informer du dépôt de la déclarat...,NaN
2409,"Code de l'urbanisme, art. A520-10",Code de l'urbanisme,Les litiges relatifs à l'assiette et à la liqu...,222393600000,32472144000000,A520-10,LEGIARTI000006814213,LEGIARTI000006814213,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie réglementaire - Arrêtés &gt; Livre V :...,NaN,NaN,A520-10,Cite moi l'article A520-10,Quelle autorité est compétente pour suivre les...,NaN
2410,"Code de l'urbanisme, art. A520-11",Code de l'urbanisme,Les dispositions des articles A. 520-4 à A. 52...,222393600000,32472144000000,A520-11,LEGIARTI000006814214,LEGIARTI000006814214,AUTONOME,VIGUEUR,...,NaN,NaN,NaN,Partie réglementaire - Arrêtés &gt; Livre V :...,NaN,NaN,A520-11,Cite moi l'article A520-11,Quelle autorité est responsable de modifier le...,NaN


In [ ]:
prompt = """
Repond à la QUESTION en utilisant le TEXTE suivant. La réponse doit être factuelle et uniquement basée sur le TEXTE.
Si la reponse ne se trouve pas dans le TEXTE, renvoi une string vide.

QUESTION:\\n {} \\n
TEXTE: \\n {}

Renvoi une reponse simple sans la question ni le texte, mais formuler avec un phrase.
"""

In [ ]:
import os
from mistralai import Mistral
from google.colab import userdata


api_key = userdata.get('MISTRAL_API_KEY')
model = "ministral-8b-2410"

client = Mistral(api_key=api_key)

def generate_response_mistralAI(prompt):

  chat_response = client.chat.complete(
      model = model,
      messages = [
          {
              "role": "user",
              "content": prompt,
          },
      ]
  )
  return chat_response.choices[0].message.content

print(generate_response_mistralAI(prompt.format(df['question_orientée'][0], df['texte'][0])))

Les collectivités publiques sont désignées comme gestionnaires et garantes du territoire français.


In [ ]:
from tqdm.notebook import tqdm
import time
import pandas as pd

def decrease_time_to_wait(time_to_wait):
  if time_to_wait > 0:
    time_to_wait -= 1
  return time_to_wait

def increase_time_to_wait(time_to_wait):
  time_to_wait += 1
  return time_to_wait

col_name = 'reponse_ministral_8b'

if col_name not in df.columns:
  df[col_name] = pd.NA

time_to_wait = 10

for i in tqdm(range(len(df))):
  done = False
  print('Iteration: ', i)
  # if not na
  if pd.notna(df.loc[i, col_name]):
    print('Already done')
    continue
  while not done:
    try:
      df.loc[i, col_name] = generate_response_mistralAI(prompt.format(df['question_orientée'][i], df['texte'][i]))
      time_to_wait = decrease_time_to_wait(time_to_wait)
      done = True
      print('Succeded! new time to wait :', time_to_wait)
    except Exception as e:
      time.sleep(time_to_wait)
      time_to_wait = increase_time_to_wait(time_to_wait)
      print('Failed! new time to wait :', time_to_wait)

# encoding for latin character
df.to_csv('df_reponse_orientees_mistral.csv', index=False, sep=';', encoding='latin1')

  0%|          | 0/2417 [00:00<?, ?it/s]

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Iteration:  750
Succeded! new time to wait : 3
Iteration:  751
Failed! new time to wait : 4
Failed! new time to wait : 5
Succeded! new time to wait : 4
Iteration:  752
Succeded! new time to wait : 3
Iteration:  753
Failed! new time to wait : 4
Failed! new time to wait : 5
Failed! new time to wait : 6
Succeded! new time to wait : 5
Iteration:  754
Succeded! new time to wait : 4
Iteration:  755
Failed! new time to wait : 5
Failed! new time to wait : 6
Succeded! new time to wait : 5
Iteration:  756
Failed! new time to wait : 6
Succeded! new time to wait : 5
Iteration:  757
Succeded! new time to wait : 4
Iteration:  758
Failed! new time to wait : 5
Succeded! new time to wait : 4
Iteration:  759
Failed! new time to wait : 5
Failed! new time to wait : 6
Succeded! new time to wait : 5
Iteration:  760
Failed! new time to wait : 6
Succeded! new time to wait : 5
Iteration:  761
Succeded! new time to wait : 4
Iteration:

In [ ]:
df[['article_id', 'question_orientée', 'reponse_ministral_8b', 'reponse_ministral_3b', 'texte']]


,article_id,question_orientée,reponse_ministral_8b,reponse_ministral_3b,texte
0,L101-1,Quels acteurs sont désignés comme gestionnaire...,Les collectivités publiques sont désignées com...,Les collectivités publiques sont les gestionna...,Le territoire français est le patrimoine commu...
1,L101-2,Quels sont les objectifs du développement dura...,Les objectifs du développement durable visés p...,Les objectifs du développement durable visés p...,Dans le respect des objectifs du développement...
2,L101-2-1,Quels sont les critères à prendre en compte po...,Les critères à prendre en compte pour atteindr...,Les critères à prendre en compte pour atteindr...,L'atteinte des objectifs mentionnés au 6° bis ...
3,L101-3,Quels sont les territoires exclus de la réglem...,Les collectivités d'outre-mer régies par l'art...,Les territoires exclus de la réglementation de...,La réglementation de l'urbanisme régit l'utili...
4,L102-1,Quelles sont les deux conditions requises pour...,Les deux conditions requises pour qu'un projet...,Le projet doit être destiné à l'une des activi...,L'autorité administrative compétente de l'Etat...
...,...,...,...,...,...
2412,A614-2,Quel est le montant de la rémunération au titr...,La rémunération au titre de leur vacation jour...,La rémunération au titre de leur vacation jour...,Les architectes-conseils et les paysagistes-co...
2413,A614-3,Quels sont les crédits sur lesquels sont imput...,Les crédits des ministères concernés.,Les dépenses de rémunération et de transport d...,Les dépenses correspondants à la rémunération ...
2414,A614-4,Quelles sont les autorités compétentes pour mo...,Les dispositions des articles A. 614-1 à A. 61...,Les autorités compétentes pour modifier les di...,Les dispositions des articles A. 614-1 à A. 61...
2415,A431-10 Annexe,Qu'est-ce que le contrôleur technique atteste ...,Le contrôleur technique atteste avoir fait con...,Le contrôleur technique atteste avoir fait con...,Attestation du contrôleur technique établissan...
